In [1]:
import os
import mne
import pickle
from src.params import MRI_PATH, SUBJ_CLEAN, RESULT_PATH
from src.utils import get_bids_file
from src.config import subjects_dir
import numpy as np
from scipy import stats as stats
from mne.stats import spatio_temporal_cluster_1samp_test, summarize_clusters_stc

## Test average condition per subject

In [19]:
####### PREPARE DATA #######

task = 'LaughterActive'
cond1_name = 'LaughReal'
cond2_name = 'LaughPosed'

####### PREPARE DATA #######

# Loop through subjects
all_subject_stcs_cond1 = []
all_subject_stcs_cond2 = []



group_all_stc_cond1 = []
group_all_stc_cond2 = []

for subj in ['01', '02']:
    _, path_epochs = get_bids_file(RESULT_PATH, task=task, subj=subj, stage="AR_epo")
    epochs = mne.read_epochs(path_epochs, verbose=None)

    # Find condition related to epoch
    cond1_epo = []
    cond2_epo = []
    
    all_stc_cond1 = []

    all_stc_cond2 = []
    for i, epo_arr in enumerate(epochs.events): 
        if epo_arr[2] == 3 :                
            cond1_epo.append(i)
        elif epo_arr[2] == 2 : 
            cond2_epo.append(i)

    cond1 = [str(nb).zfill(3) for nb in cond1_epo]
    cond2 = [str(nb).zfill(3) for nb in cond2_epo]
    print('Cond1', len(cond1))
    print('Cond2', len(cond2))
    
    # Find morph data of the same condition
    for epo1 in cond1 : # Cond1
        stc_path_cond1 = os.path.join(MRI_PATH, f'sub-{subj}', 'morph',
                                    f'{epo1}_MNE_morph-stc.h5')

        stc_cond1 = mne.read_source_estimate(stc_path_cond1)
        all_stc_cond1.append(stc_cond1)

    for epo2 in cond2 : # Cond2
        stc_path_cond2 = os.path.join(MRI_PATH, f'sub-{subj}', 'morph',
                                    f'{epo2}_MNE_morph-stc.h5')

        stc_cond2 = mne.read_source_estimate(stc_path_cond2)
        all_stc_cond2.append(stc_cond2)

    grand_ave_cond1 = np.mean(all_stc_cond1) 
    grand_ave_cond2 = np.mean(all_stc_cond2) 
    
    print(grand_ave_cond1.data)

"""    print(len(all_stc_cond1)) 
    print(len(all_stc_cond2)) 
    
    print(stc_cond1.data.shape)

    ######## AVERAGE SIGNAL PER CONDITION FOR 1 SUBJ #########
    for subject_stcs1 in all_stc_cond1 : 

        # Average the SourceEstimates for the current subject
        avg_data_cond1 = np.mean([s.data for s in all_stc_cond1], axis=0)
    
        # Append to the list for all subjects
        all_subject_stcs_cond1.append(avg_data_cond1)

    print('All subj cond1 :', len(all_subject_stcs_cond1)) # Should be n_subj

    for subject_stcs2 in all_stc_cond2 : 

        # Average the SourceEstimates for the current subject
        avg_data_cond2 = np.mean([s.data for s in all_stc_cond2], axis=0)

        # Append to the list for all subjects
        all_subject_stcs_cond2.append(avg_data_cond2)

    print('All subj cond2 :', len(all_subject_stcs_cond2)) # Should be n_subj
    
######## AVERAGE ARRAYS ACROSS SUBJECTS #########
 # Average all subject all epochs CONDITION 1
group_avg_data_cond1 = np.mean(all_subject_stcs_cond1, axis=0)

# Compute Source Estimate with average value - Cond 1
group_avg_stc_cond1 = mne.SourceEstimate(group_avg_data_cond1, 
                                         vertices=stc_cond1.vertices, 
                                         tmin=stc_cond1.tmin, 
                                         tstep=stc_cond1.tstep, 
                                         subject=stc_cond1.subject)

# Average all subject all epochs CONDITION 2
group_avg_data_cond2 = np.mean(all_subject_stcs_cond2, axis=0)

# Compute Source Estimate with average value - Cond 2
group_avg_stc_cond2 = mne.SourceEstimate(group_avg_data_cond2, 
                                         vertices=stc_cond2.vertices, 
                                         tmin=stc_cond2.tmin, 
                                         tstep=stc_cond2.tstep, 
                                         subject=stc_cond2.subject)

group_all_stc_cond1.append(group_avg_stc_cond1.data)
group_all_stc_cond2.append(group_avg_stc_cond2.data)


print(np.array(group_all_stc_cond1).shape)
print(np.array(group_all_stc_cond2).shape)

# contrast = np.array(group_all_stc_cond1)-np.array(group_all_stc_cond2) """

Reading /home/claraelk/scratch/laughter_data/results/meg/reports/sub-01/sub-01_task-LaughterActive_run-all_AR_epo.fif ...
    Found the data of interest:
        t =    -500.00 ...    1500.00 ms
        0 CTF compensation matrices available
Adding metadata with 7 columns
296 matching events found
No baseline correction applied
0 projection items activated
Cond1 73
Cond2 74
[[5.69848951e-11 5.96709755e-11 6.04139606e-11 ... 5.78296795e-11
  5.89044321e-11 5.81591304e-11]
 [7.04388674e-11 6.81827466e-11 6.51834231e-11 ... 6.05369134e-11
  6.38491105e-11 6.63099388e-11]
 [4.16683818e-11 4.30833737e-11 4.33898840e-11 ... 4.20610316e-11
  4.33418436e-11 4.42929466e-11]
 ...
 [1.16494224e-10 1.18782809e-10 1.25432887e-10 ... 1.16477158e-10
  1.16553588e-10 1.20627702e-10]
 [1.18722068e-10 1.19620771e-10 1.24822550e-10 ... 1.14858708e-10
  1.14436696e-10 1.17713666e-10]
 [1.15720023e-10 1.14798506e-10 1.17059149e-10 ... 1.08920257e-10
  1.08389159e-10 1.10971285e-10]]
Reading /home/claraelk/s

"    print(len(all_stc_cond1)) \n    print(len(all_stc_cond2)) \n    \n    print(stc_cond1.data.shape)\n\n    ######## AVERAGE SIGNAL PER CONDITION FOR 1 SUBJ #########\n    for subject_stcs1 in all_stc_cond1 : \n\n        # Average the SourceEstimates for the current subject\n        avg_data_cond1 = np.mean([s.data for s in all_stc_cond1], axis=0)\n    \n        # Append to the list for all subjects\n        all_subject_stcs_cond1.append(avg_data_cond1)\n\n    print('All subj cond1 :', len(all_subject_stcs_cond1)) # Should be n_subj\n\n    for subject_stcs2 in all_stc_cond2 : \n\n        # Average the SourceEstimates for the current subject\n        avg_data_cond2 = np.mean([s.data for s in all_stc_cond2], axis=0)\n\n        # Append to the list for all subjects\n        all_subject_stcs_cond2.append(avg_data_cond2)\n\n    print('All subj cond2 :', len(all_subject_stcs_cond2)) # Should be n_subj\n    \n######## AVERAGE ARRAYS ACROSS SUBJECTS #########\n # Average all subject all epoc

In [4]:
task = 'LaughterActive'
cond1_name = 'LaughReal'
cond2_name = 'LaughPosed'

group_all_stc_cond1 = []
group_all_stc_cond2 = []

for subj in SUBJ_CLEAN :
    filename_cond1, _ = get_bids_file(RESULT_PATH, subj=subj, task=task, stage='erp-morph-src', condition=cond1_name)
    filename_cond2, _ = get_bids_file(RESULT_PATH, subj=subj, task=task, stage='erp-morph-src', condition=cond2_name)

    save_cond1 = os.path.join(MRI_PATH, f'sub-{subj}', filename_cond1[:-4])
    save_cond2 = os.path.join(MRI_PATH, f'sub-{subj}', filename_cond2[:-4])

    stc_cond1 = mne.read_source_estimate(save_cond1)
    stc_cond2 = mne.read_source_estimate(save_cond2)

    group_all_stc_cond1.append(stc_cond1.data)
    group_all_stc_cond2.append(stc_cond2.data)

print(np.array(group_all_stc_cond1).shape)
print(np.array(group_all_stc_cond2).shape)

(27, 20484, 2401)
(27, 20484, 2401)


## Average subject file

In [5]:
task = 'LaughterActive'
cond1_name = 'LaughReal'
cond2_name = 'LaughPosed'

all_subj_stc_cond1 = []
all_subj_stc_cond2 = []

for subj in SUBJ_CLEAN: 
    print(f'Compute subj-{subj}')

    filename_cond1, _ = get_bids_file(RESULT_PATH, subj=subj, task=task, stage='erp-morph-src', condition=cond1_name)
    filename_cond2, _ = get_bids_file(RESULT_PATH, subj=subj, task=task, stage='erp-morph-src', condition=cond2_name)
 
    filename_cond1 = filename_cond1[:-4]
    filename_cond2 = filename_cond2[:-4]

    save_cond1 = os.path.join(MRI_PATH, f'sub-{subj}', filename_cond1)
    save_cond2 = os.path.join(MRI_PATH, f'sub-{subj}', filename_cond2)
    
    print('--> Read Source Estimate')
    stc_1 = mne.read_source_estimate(save_cond1)
    stc_2 = mne.read_source_estimate(save_cond2)
    
    all_subj_stc_cond1.append(stc_1)
    all_subj_stc_cond2.append(stc_2)

    
print('Len condition 1 :', len(all_subj_stc_cond1)) # Length should be n_subj
print('Len condition 2 :', len(all_subj_stc_cond2)) # Length should be n_subj

# Average subject together per condition
print('Average per condition')
grand_ave_cond1 = np.mean(all_subj_stc_cond1)
grand_ave_cond2 = np.mean(all_subj_stc_cond2)

print('Average --> Done')
print(grand_ave_cond1)
print(grand_ave_cond2)

print('Save file')
filename_all_subj_cond1, _ = get_bids_file(RESULT_PATH, subj='all', task=task, stage='erp-morph-src', condition=cond1_name)
filename_all_subj_cond2, _ = get_bids_file(RESULT_PATH, subj='all', task=task, stage='erp-morph-src', condition=cond2_name)

filename_all_subj_cond1 = filename_all_subj_cond1[:-4]
filename_all_subj_cond2 = filename_all_subj_cond2[:-4]

save_all_cond1 = os.path.join(MRI_PATH, 'sub-all', filename_all_subj_cond1[:-4])
save_all_cond2 = os.path.join(MRI_PATH, 'sub-all', filename_all_subj_cond2[:-4])

grand_ave_cond1.save(save_all_cond1, ftype='h5', overwrite=True)
grand_ave_cond2.save(save_all_cond2, ftype='h5', overwrite=True)

Compute subj-01
--> Read Source Estimate
Compute subj-02
--> Read Source Estimate
Compute subj-03
--> Read Source Estimate
Compute subj-04
--> Read Source Estimate
Compute subj-05
--> Read Source Estimate
Compute subj-06
--> Read Source Estimate
Compute subj-07
--> Read Source Estimate
Compute subj-08
--> Read Source Estimate
Compute subj-09
--> Read Source Estimate
Compute subj-10
--> Read Source Estimate
Compute subj-11
--> Read Source Estimate
Compute subj-12
--> Read Source Estimate
Compute subj-13
--> Read Source Estimate
Compute subj-14
--> Read Source Estimate
Compute subj-15
--> Read Source Estimate
Compute subj-16
--> Read Source Estimate
Compute subj-18
--> Read Source Estimate
Compute subj-19
--> Read Source Estimate
Compute subj-20
--> Read Source Estimate
Compute subj-21
--> Read Source Estimate
Compute subj-22
--> Read Source Estimate
Compute subj-23
--> Read Source Estimate
Compute subj-24
--> Read Source Estimate
Compute subj-25
--> Read Source Estimate
Compute subj-28


In [4]:
task = 'LaughterActive'
cond1_name = 'LaughReal'
cond2_name = 'LaughPosed'

filename_all_subj_cond1, _ = get_bids_file(RESULT_PATH, subj='all', task=task, stage='erp-morph-src', condition=cond1_name)
filename_all_subj_cond2, _ = get_bids_file(RESULT_PATH, subj='all', task=task, stage='erp-morph-src', condition=cond2_name)
print(filename_all_subj_cond1)


filename_all_subj_cond1 = filename_all_subj_cond1[:-4]
filename_all_subj_cond2 = filename_all_subj_cond2[:-4]

save_all_cond1 = os.path.join(MRI_PATH, 'sub-all', filename_all_subj_cond1[:-4])
save_all_cond2 = os.path.join(MRI_PATH, 'sub-all', filename_all_subj_cond2[:-4])

print(save_all_cond1)

sub-all_task-LaughterActive_run-all_cond-LaughReal_erp-morph-src.fif
/home/claraelk/scratch/laughter_data/results/mri/sub-all/sub-all_task-LaughterActive_run-all_cond-LaughReal_erp-morph


## Import data averaged per subject for each condition

In [10]:
contrast_data = []
group_all_stc_cond1 = []
group_all_stc_cond2 =[]

task = 'LaughterActive'
cond1_name = 'LaughReal'
cond2_name = 'LaughPosed'
conditions = cond1_name +'-'+ cond2_name

for subj in ['01'] :
    print('Loading subj', subj)
    filename_contrast, _ = get_bids_file(RESULT_PATH, subj=subj, task=task, stage='erp-morph-contrast', condition=conditions)
    filename_cond1, _ = get_bids_file(RESULT_PATH, subj=subj, task=task, stage='erp-morph-contrast', condition=cond1_name)
    filename_cond2, _ = get_bids_file(RESULT_PATH, subj=subj, task=task, stage='erp-morph-contrast', condition=cond2_name)

    save_contrasts = os.path.join(MRI_PATH, 'sub-all', filename_contrast)

    save_cond1 = os.path.join(MRI_PATH, 'sub-all', filename_cond1)
    save_cond2 = os.path.join(MRI_PATH, 'sub-all', filename_cond2)

    with open(save_contrasts, 'rb') as f:
        contrast_subj = pickle.load(f)  

    with open(save_cond1, 'rb') as f:
        group_all_stc_cond1_subj = pickle.load(f)  

    with open(save_cond2, 'rb') as f:
        group_all_stc_cond2_subj = pickle.load(f) 
   
    contrast_data.append(contrast_subj)
    group_all_stc_cond1.append(group_all_stc_cond1_subj[0])
    group_all_stc_cond2.append(group_all_stc_cond2_subj[0])

Loading subj 01


In [8]:
stc_path_cond1 = os.path.join(MRI_PATH, f'sub-01', 'morph',
                            f'000_MNE_morph-stc.h5')

stc_cond1 = mne.read_source_estimate(stc_path_cond1)

data_stc_group1 = np.mean(np.array(group_all_stc_cond1), axis=0)
test_cond1 = mne.SourceEstimate(data_stc_group1, 
                                vertices=stc_cond1.vertices, 
                                tmin=stc_cond1.tmin, 
                                tstep=stc_cond1.tstep, 
                                subject='sub-all')

data_stc_group2 = np.mean(np.array(group_all_stc_cond2), axis=0)
test_cond2 = mne.SourceEstimate(data_stc_group2, 
                                vertices=stc_cond1.vertices, 
                                tmin=stc_cond1.tmin, 
                                tstep=stc_cond1.tstep, 
                                subject='sub-all')

In [5]:
grand_ave = np.mean([test_cond1, test_cond2]) #, evoked3]).data

filname_ave1 = f'sub-all_task-LaughterActive_run-all_cond-{cond1_name}_erp-morph-ratio-stc.h5'
path_ave1 = os.path.join(MRI_PATH, 'sub-all', filname_ave1)

grand_ave_cond1.save(path_ave1, ftype='h5')

filname_ave2 = f'sub-all_task-LaughterActive_run-all_cond-{cond2_name}_erp-morph-ratio-stc.h5'
path_ave2 = os.path.join(MRI_PATH, 'sub-all', filname_ave2)

grand_ave_cond2.save(path_ave2, ftype='h5')

[done]
[done]


In [7]:
ratio = (np.array(grand_ave_cond1.data)-np.array(grand_ave_cond2.data))/np.array(grand_ave_cond2.data)
ratio_ave = np.mean(ratio, axis=0)
print(ratio_ave.shape)

#ratio_name, _ = get_bids_file(RESULT_PATH, subj='all', task=task, stage='erp-morph-ratio', condition=conditions)
ratio_name = 'sub-all_task-LaughterActive_run-all_cond-LaughReal-LaughPosed_erp-morph-ratio_ave.pkl'
ratio_path = os.path.join(MRI_PATH, 'sub-all', ratio_name)

with open(ratio_path, 'wb') as f:
    pickle.dump(ratio_ave, f) 

In [11]:
# Save Average of the ratio in STC
stc_path_cond1 = os.path.join(MRI_PATH, f'sub-01', 'morph',
                            f'000_MNE_morph-stc.h5')

stc_cond1 = mne.read_source_estimate(stc_path_cond1)

avg_stc_ratio = mne.SourceEstimate(ratio, 
                                vertices=stc_cond1.vertices, 
                                tmin=stc_cond1.tmin, 
                                tstep=stc_cond1.tstep, 
                                subject='sub-all')

ratio_ave_name ='sub-all_task-LaughterActive_run-all_cond-LaughReal-LaughPosed_erp-morph-ratio-stc_grand-ave.h5'
ratio_ave_path = os.path.join(MRI_PATH, 'sub-all', ratio_ave_name)

avg_stc_ratio.save(ratio_ave_path, ftype='h5')

[done]


In [15]:
nan_mask = np.isnan(contrast)
if np.any(nan_mask):
    print("Warning: NaN values found in the data. Please address and rerun the analysis.")

In [29]:
print("Min value:", np.min(contrast))
print("Max value:", np.max(contrast))


Min value: -3.0816553290181176e-10
Max value: 2.373586805991708e-10


## Compute adjacency

In [18]:
# Read the source space we are morphing to
src_fname = '/home/claraelk/scratch/laughter_data/results/mri/anat/subjects/fsaverage/bem/fsaverage-5-src.fif'
fname_inv = os.path.join(MRI_PATH, 'sub-01', 'src', 'sub-01-mag-oct6-inv.fif')
inverse_operator = mne.minimum_norm.read_inverse_operator(fname_inv)

src = mne.read_source_spaces(src_fname)
fsave_vertices = [s["vertno"] for s in src]
morph_mat = mne.compute_source_morph(
    src=inverse_operator["src"],
    subject_to="fsaverage",
    spacing=fsave_vertices,
    subjects_dir=subjects_dir,
).morph_mat

n_vertices_fsave = morph_mat.shape[0]


Reading inverse operator decomposition from /home/claraelk/scratch/laughter_data/results/mri/sub-01/src/sub-01-mag-oct6-inv.fif...
    Reading inverse operator info...
    [done]
    Reading inverse operator decomposition...
    [done]
    270 x 270 full covariance (kind = 1) found.
    Noise covariance matrix read.
    24585 x 24585 diagonal covariance (kind = 2) found.
    Source covariance matrix read.
    24585 x 24585 diagonal covariance (kind = 6) found.
    Orientation priors read.
    24585 x 24585 diagonal covariance (kind = 5) found.
    Depth priors read.
    Did not find the desired covariance matrix (kind = 3)
    Reading a source space...
    Computing patch statistics...
    Patch information added...
    Distance information added...
    [done]
    Reading a source space...
    Computing patch statistics...
    Patch information added...
    Distance information added...
    [done]
    2 source spaces read
    Source spaces transformed to the inverse solution coordinate

'\n# We have to change the shape for the dot() to work properly\nX = X.reshape(n_vertices_sample, n_times * n_subjects * 2)\nprint("Morphing data.")\nX = morph_mat.dot(X)  # morph_mat is a sparse matrix\nX = X.reshape(n_vertices_fsave, n_times, n_subjects, 2)\n'

In [ ]:
n_subjects = 27


# We have to change the shape for the dot() to work properly
group_cond1 = np.array(group_all_stc_cond1)
group_cond1 = group_cond1.reshape(n_vertices_sample, n_times * n_subjects)
print("Morphing data.")
group_cond1 = morph_mat.dot(group_cond1)  # morph_mat is a sparse matrix
group_cond1 = group_cond1.reshape(n_vertices_fsave, n_times, n_subjects)

In [30]:
file_src = 'sub-02-mag-oct6-inv.fif'
path_src = os.path.join(MRI_PATH, 'sub-02', 'src', file_src)

In [32]:
src_fname = '/home/claraelk/scratch/laughter_data/results/mri/anat/subjects/fsaverage/bem/fsaverage-5-src.fif'

src = mne.read_source_spaces(path_src)
print(type(src))

    Reading a source space...
    Computing patch statistics...
    Patch information added...
    Distance information added...
    [done]
    Reading a source space...
    Computing patch statistics...
    Patch information added...
    Distance information added...
    [done]
    2 source spaces read
<class 'mne.source_space.SourceSpaces'>


In [36]:
# Find adjacency
print("Computing adjacency.")
adjacency = mne.spatial_src_adjacency(src, dist=1)

Computing adjacency.
-- number of adjacent vertices : 8195


## Compute statistics

In [38]:
# Note that X needs to be a multi-dimensional array of shape
# observations (subjects) × time × space, so we permute dimensions
X = np.transpose(contrast, [0, 2, 1])
print(X.shape)
n_subjects = len(contrast)
print(n_subjects)

# Here we set a cluster forming threshold based on a p-value for
# the cluster based permutation test.
# We use a two-tailed threshold, the "1 - p_threshold" is needed
# because for two-tailed tests we must specify a positive threshold.
p_threshold = 0.001
df = n_subjects - 1  # degrees of freedom for the test
t_threshold = stats.distributions.t.ppf(1 - p_threshold / 2, df=df)

# Now let's actually do the clustering. This can take a long time...
print("Clustering.")
T_obs, clusters, cluster_p_values, H0 = clu = spatio_temporal_cluster_1samp_test(
    X,
    adjacency=adjacency,
    n_jobs=-1,
    threshold=None,
    buffer_size=None,
    verbose=True,
    n_permutations=1024,
    tail=0,
    step_down_p = 0.05,
    check_disjoint=True,
    out_type='indices'
)

(27, 2401, 8195)
27
Clustering.
Using a threshold of 2.055529
stat_fun(H1): min=-6.895321 max=6.250080
2 disjoint adjacency sets found
Running initial clustering …
Found 8 clusters


  0%|          | Permuting : 0/1023 [00:00<?,       ?it/s]

KeyboardInterrupt: 

In [3]:
print('Loading cluster')
task = 'LaughterActive'
cond1_name = 'LaughReal'
cond2_name = 'LaughPosed'

conditions = f'{cond1_name}-{cond2_name}'

save_cluster_stats, _ = get_bids_file(RESULT_PATH, stage = "erp-source", task=task, measure="Ttest-clusters-src", condition = conditions)

path_save_cluster = os.path.join(MRI_PATH, 'sub-all', save_cluster_stats)

with open(path_save_cluster, 'rb') as f:
    clu = pickle.load(f)  

Loading cluster


In [4]:
path_save_cluster

'/home/claraelk/scratch/laughter_data/results/mri/sub-all/sub-all_task-LaughterActive_run-all_cond-LaughReal-LaughPosed_meas-Ttest-clusters-src_erp-source.fif'

In [11]:
T_obs, clusters, cluster_p_values, H0 = clu 

# Select the clusters that are statistically significant at p < 0.05
good_clusters_idx = np.where(cluster_p_values < 0.05)[0]
good_clusters = [clusters[idx] for idx in good_clusters_idx]
print(good_clusters)

[]


In [13]:
cluster_p_values == 1

array([ True,  True,  True, ...,  True,  True,  True])

## Test

In [2]:
task = 'LaughterActive'
cond1_name = 'LaughReal'
cond2_name = 'LaughPosed'

contrast_data = []
group_all_stc_cond1 = []
group_all_stc_cond2 =[]
conditions = cond1_name + "-" + cond2_name

crop = True
tmin = 0.0
tmax = 1.5

##### Import data
for subj in SUBJ_CLEAN :
    filename_cond1, _ = get_bids_file(RESULT_PATH, subj=subj, task=task, stage='erp-morph-src', condition=cond1_name)
    filename_cond2, _ = get_bids_file(RESULT_PATH, subj=subj, task=task, stage='erp-morph-src', condition=cond2_name)

    save_cond1 = os.path.join(MRI_PATH, f'sub-{subj}', filename_cond1[:-4])
    save_cond2 = os.path.join(MRI_PATH, f'sub-{subj}', filename_cond2[:-4])

    stc_cond1 = mne.read_source_estimate(save_cond1)
    stc_cond2 = mne.read_source_estimate(save_cond2)

    if crop == True : 
        stc_cond1.crop(tmin, tmax)
        stc_cond2.crop(tmin, tmax)

    group_all_stc_cond1.append(stc_cond1.data)
    group_all_stc_cond2.append(stc_cond2.data)

###### Compute contrast
contrast = np.array(group_all_stc_cond1)-np.array(group_all_stc_cond2) 
print('Shape contrast :', contrast.shape)

###### Compute adjacency #######
src_fname = os.path.join(MRI_PATH, 'anat/subjects', 'fsaverage/bem', 'fsaverage-5-src.fif')

src = mne.read_source_spaces(src_fname)
print(type(src))

# Find adjacency
print("Computing adjacency.")
adjacency = mne.spatial_src_adjacency(src, dist=None)
print(adjacency.shape)

# Note that X needs to be a multi-dimensional array of shape
# observations (subjects) × time × space, so we permute dimensions
X = np.transpose(contrast, [0, 2, 1])
print(X.shape)
n_subjects = len(contrast)
print(n_subjects)


########### STATISTICAL ANALYSIS ##########

# Here we set a cluster forming threshold based on a p-value for
# the cluster based permutation test.
# We use a two-tailed threshold, the "1 - p_threshold" is needed
# because for two-tailed tests we must specify a positive threshold.
p_threshold = 0.001
df = n_subjects - 1  # degrees of freedom for the test
t_threshold = stats.distributions.t.ppf(1 - p_threshold / 2, df=df)
n_perm = 'all'

# Now let's actually do the clustering. This can take a long time...
print("Clustering.")
T_obs, clusters, cluster_p_values, H0 = clu = spatio_temporal_cluster_1samp_test(
    X,
    adjacency=adjacency,
    n_jobs=-1,
    threshold=None,
    buffer_size=None,
    verbose=True,
    n_permutations= n_perm,
    tail=0,
    check_disjoint=False,
    out_type='indices'
)

# Select the clusters that are statistically significant at p < 0.05
good_clusters_idx = np.where(cluster_p_values < 0.05)[0]
good_clusters = [clusters[idx] for idx in good_clusters_idx]
print(good_clusters)

print('Saving cluster')
if crop == False : 
    save_cluster_stats, _ = get_bids_file(RESULT_PATH, stage = "erp-src", task=task, 
                                          measure=f"Ttest-clusters-morph-perm-{n_perm}", 
                                          condition = conditions)

else :
    save_cluster_stats, _ = get_bids_file(RESULT_PATH, stage = "erp-src", task=task, 
                                          measure=f"Ttest-clusters-morph-perm-{n_perm}_time{tmin}-{tmax}", 
                                          condition = conditions)

path_save_cluster = os.path.join(MRI_PATH, 'sub-all', save_cluster_stats)

#with open(path_save_cluster, 'wb') as f:
#    pickle.dump(clu, f)  

Shape contrast : (27, 20484, 1801)
    Reading a source space...
    Computing patch statistics...
    Patch information added...
    Distance information added...
    [done]
    Reading a source space...
    Computing patch statistics...
    Patch information added...
    Distance information added...
    [done]
    2 source spaces read
<class 'mne.source_space.SourceSpaces'>
Computing adjacency.
-- number of adjacent vertices : 20484
(20484, 20484)
(27, 1801, 20484)
27
Clustering.
Using a threshold of 2.055529
stat_fun(H1): min=-6.911755 max=6.263853
Running initial clustering …
Found 28091 clusters


KeyboardInterrupt: 